In [97]:
from __future__ import print_function
import nltk
import os

# Setup nltk corpora path
nltk_path = os.sep.join([os.environ['HOME'], 'nltk_data'])
nltk.data.path.insert(0, nltk_path)

In [98]:
import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit
import string
import re

#import snips as snp  # my snippets
#snp.prettyplot(matplotlib)  # my aesthetic preferences for plotting
%matplotlib inline

In [99]:
pkl_filename_master = "df_games.pkl"
with open(pkl_filename_master, 'rb') as picklefile: 
    df_games = pickle.load(picklefile)

In [100]:
test_df = df_games.copy()

In [101]:
test_df = test_df.drop([ u'id', u'year_released', u'geek_rating', u'average_rating', 
       u'nrate', u'pic_url', u'url', u'index1', u'min_players', u'max_players'], axis =1)

In [102]:
test_df["all_text"] = test_df["attributes"].map(str) + test_df["description"]

In [103]:
test_df["all_text_2"] = test_df["category"].map(str) + test_df["all_text"]

In [104]:
import string
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [133]:
stop_words = set(nltk.corpus.stopwords.words('english'))
#stop_words.update(['pizza','place','slice','good','food','great','like','one','get','back','really','new','york'])
stop_words.update(['good','version',"rdquo", "ldquo",'u','sushi', 'munchkin','expans','small','carcassonn','amp', 'good','wonder','Dominion', 'Catan', u'take', u'ndash',u'may', u'use', u'play', u'many', u'turn',u'take', u'use', u'make', u'rsquo', u'edition', u'know', u'x', u'warhammer',u'pandemic', u'kosmos', u'splendor', u'introduction',u'battlestar', u'galactica', u'chess', u'gloom', u'description', u'publisher', u'stuff', u'usually', u'only', u'try', u'die', u'taking', u'used', u'time', u'victory', u'different', u'must', u'also',u'dominion', u'carcassonne' ,u'carcassonne', u'end', u'used', u'only', u'taking', u'editions', u'catan', u'get', u'first', u'expansion', u'expansions', u'ticket',  u'ticket', u'ride', u'arkham', u'two', u'end', u'three', u'taking', u'number', u'win', u'carcassonne', u'one', u'used', u'new', u'play',u'player', u'players',u'playing', u'include', u'includes',u'munchkin', u'carcassonne', u"card", u'cards',u'board',u'boards',u'rule',u'rules',u'game',u'games',u'point',u'points',u"'s",u'score',u'scores',u"quot"])

In [134]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(str(text)) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(str(token))
    stems = [str(stemmer.stem(t)) for t in filtered_tokens]
    #print(stems)
    return stems


In [135]:

titles = test_df['name'].tolist()
titles_str =  [s.encode('ascii','ignore') for s in titles]
titles_str


['Catan',
 'Carcassonne',
 'Pandemic',
 'Dominion',
 '7 Wonders',
 'Ticket to Ride',
 'Agricola',
 'Puerto Rico',
 'Small World',
 'Power Grid',
 'Citadels',
 'King of Tokyo',
 'Love Letter',
 'Ticket to Ride: Europe',
 'Race for the Galaxy',
 'Dixit',
 'Stone Age',
 'Arkham Horror',
 'Codenames',
 'Splendor',
 'Lords of Waterdeep',
 'Munchkin',
 'Bohnanza',
 'Twilight Struggle',
 'Forbidden Island',
 'Lost Cities',
 'The Resistance',
 'Battlestar Galactica',
 'The Castles of Burgundy',
 'Terra Mystica',
 'Hanabi',
 'Dominion: Intrigue',
 'Dead of Winter: A Crossroads Game',
 'Magic: The Gathering',
 'Risk',
 'Betrayal at House on the Hill',
 'Caylus',
 'Alhambra',
 'Coup',
 'Galaxy Trucker',
 '7 Wonders Duel',
 'Android: Netrunner',
 'Eclipse',
 'Shadows over Camelot',
 'Chess',
 'Tigris & Euphrates',
 'Hive',
 'Takenoko',
 'BANG!',
 'Monopoly',
 'RoboRally',
 "Memoir '44",
 'Scrabble',
 'Le Havre',
 'Cosmic Encounter',
 'San Juan',
 'Patchwork',
 'Star Realms',
 'El Grande',
 'Jaipur

In [136]:
stop_words.update(titles_str)
#stop_words.update(tokenize_and_stem(str(titles)))
stop_words = [s.encode('ascii','ignore') for s in stop_words]   #removes unicode and changes to str. Remove if a problem. 

stop_words = set(stop_words)
stop_words.update(["rdquo", "ldquo"])
stop_words = set(stop_words)

In [137]:
#stop_words = set(stop_words)
stop_words
#sorted(stop_words,reverse=True)

{'Amerigo',
 'Show Manager',
 'Power Grid: Brazil/Spain & Portugal',
 'Terra Nova',
 'Empire Builder',
 'galactica',
 'Seeland',
 'Eight-Minute Empire: Legends',
 'Get Bit!',
 'Toledo',
 'David & Goliath',
 'Stockpile',
 'Rum & Pirates',
 'Livingstone',
 'Vinci',
 'BattleTech',
 'Mage Knight Board Game: The Lost Legion',
 'Space Hulk (third edition)',
 "That's Life!",
 '7 Wonders Duel: Pantheon',
 'to',
 'Pathfinder Adventure Card Game: Rise of the Runelords  Character Add-On Deck',
 'under',
 'Cadwallon: City of Thieves',
 'Hotel Tycoon',
 'Cutthroat Caverns',
 'Arkham Horror',
 'very',
 'Revolution!',
 'Battle Cry',
 'Bora Bora',
 'Through the Ages: A Story of Civilization',
 'Tournay',
 'Coin Age',
 'Galaxy Trucker',
 'Rallyman',
 'Eminent Domain',
 'Tomb',
 'New York 1901',
 'Steam',
 "Can't Stop",
 'AquaSphere',
 'Agricola: Farmers of the Moor',
 'Pergamon',
 'War of the Ring: Battles of the Third Age',
 'Lewis & Clark',
 'Space Hulk: Death Angel  The Card Game',
 'Archipelago',
 

In [138]:
"ldquo" in stop_words

True

In [139]:
with open('stop_words.pkl', 'wb') as f:
    pickle.dump(stop_words, f)

Add titles as stopwords:

In [63]:
stop_words.update(tokenize_and_stem(str(titles_str)))

In [64]:
sorted(stop_words, reverse=True)

['zrtz',
 'zooloretto',
 'zoo',
 'zombicid',
 'zombi',
 'zimbabw',
 'zhanguo',
 'zeus',
 'zendo',
 'zavandor',
 'yspahan',
 'ys',
 'yourselves',
 'yourself',
 'yours',
 'your',
 'you',
 'york',
 'yomi',
 'yinsh',
 'yggdrasil',
 'yellow',
 'yedo',
 'year',
 'yard',
 'yahtze',
 'y-wing',
 'y',
 'xiv',
 'xidit',
 'xiangqi',
 'xia',
 'xenoshyft',
 'xcom',
 'x-wing',
 'x-men',
 'x',
 'wyrm',
 'wyatt',
 'wrfel',
 'wrath',
 'wouldn',
 'worldspann',
 'world',
 'word',
 'wooli',
 'wooden',
 'wood',
 'wonder',
 'won',
 'wizard',
 'wiz-war',
 'without',
 'with',
 'witcher',
 'witch',
 'wit',
 'winter/desert',
 'winter',
 'winner',
 'wing',
 'wind',
 'win',
 'will',
 'wildlif',
 'wilder',
 'wick',
 'why',
 'whom',
 'who',
 'whitechapel',
 'white',
 'while',
 'which',
 'where',
 'when',
 'what',
 'westero',
 'western',
 'west',
 'werewolv',
 'werewolf',
 'weren',
 'were',
 'well',
 'welcom',
 'web',
 'wealth',
 'we',
 'way',
 'waterwork',
 'waterdeep',
 'watch',
 'wast',
 'wasn',
 'washington',
 'w

In [65]:
with open('stop_words_stemmed_titles.pkl', 'wb') as f:
    pickle.dump(stop_words, f)